## Alcholic Anonymous News articles extraction from Daily Sun. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.dailysun.co.za'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

11 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
            except:
                headlines.append(article.title.strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[name="description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="cXenseParse:tss-byline"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[name="cXenseParse:recs:publishtime"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('div.content').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 54.5 sdailysun.co.za/Search/Lost?p=12s-bust-booze-thieves-20160225eck-20181002ile=truerue


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

11 11 11 11 11 11 11 11


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})
#tbl = tbl.dropna()
tbl.to_csv('Daily_Sun.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries
0,BOOZE RUINS LIVES!,“THE wake-up call for me should have been the ...,Jabu Kumalo,2018-08-15T14:30:02.000Z,“THE wake-up call for me should have been the ...,"[drinking, hit, hat, imbizo, involved, decided...",He told Daily Sun he sometimes couldn’t believ...
1,"ALCOHOLICS, IT'S NEVER TOO LATE!",IF YOU have a drinking problem or you know som...,Jabu Kumalo,2017-10-11T15:30:03.000Z,IF YOU have a drinking problem or you know som...,"[problem, drinking, soweto, group, west, late,...",IF YOU have a drinking problem or you know som...
2,my girlfriend drinks too much,WORRIED LOVER writes: My girlfriend drinks too...,None,None,WORRIED LOVER writes:My girlfriend drinks too ...,"[kick, im, drinks, fear, love, writesmy, try, ...",WORRIED LOVER writes:My girlfriend drinks too ...
3,NO BOOZE FOR THESE TWO!,SUNDAY was a joyful day as a husband and wife ...,Jabu Kumalo,2018-10-03T18:00:10.000Z,SUNDAY was a joyful day as a husband and wife ...,"[times, soweto, yearsshe, yearsthe, imbizo, wi...",SUNDAY was a joyful day as a husband and wife ...
4,BABY MAMA LIKES BOOZE,Dear MizzB.,None,None,Dear MizzBI’m a 24-year-old man and I have a p...,"[drink, times, way, stop, baby, think, things,...",I don’t even drink and I’ve spoken to her many...
